<a href="https://colab.research.google.com/github/grayfactory/ProjectSc3-MovieRec/blob/master/ds_cs_N424a_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<img src='https://user-images.githubusercontent.com/6457691/90080969-0f758d00-dd47-11ea-8191-fa12fd2054a7.png' width = '200' align = 'right'>

## *DATA SCIENCE / SECTION 4 / SPRINT 1 / Assignment 4*

---
# N414. 신경망과 학습에 관련된 파라미터 튜닝 (HyperTune)



## 실전 연습과제

다음 통신사 고객 이탈(Churn) 데이터셋에서 정확도를 조정해보는 파라미터 학습을 진행해보겠습니다 : [다운로드](https://ds-lecture-data.s3.ap-northeast-2.amazonaws.com/telecom/TelcomCustomer.csv)

## 진행방식

- 데이터를 다운로드 받고 읽어옴(load)
- 데이터 클리닝을 진행 (필수는 아니지만 추천)
- Keras MLP model을 만들고, 학습 진행
- Hyperparameter 튜닝 진행:
 - batch_size
 - training epochs
 - optimizer
 - learning rate (optimizer에 따라서 해당되면)
 - momentum (optimizer에 따라서 해당되면)
 - activation functions
 - network weight initialization
 - dropout regularization
 - number of neurons in the hidden layer
 
하이퍼 파라미터의 초기 패스를 위해 그리드 검색 및 교차 검증을 사용할 수 있어야 합니다. 

실제 큰 통신회사의 데이터이기 때문에 최대한 정확하게 파악해 보십시오! 


# 데이터 전처리 

### 문항 1) 내 로컬 파일을 colab에 업로드하기

충분한 GPU를 가지고 있다면, 쉽게 문제를 해결할 수 있겠지만, 제한된 자원에서 충분한 GPU를 제공받지 못할 지 모릅니다. 이럴 때에는 딥러닝 커뮤니티에 물어볼 수도 있겠지만, 가성비 좋은 Colab의 GPU를 이용해서 실제 GPU사용량을 예측할 수 있다면 좋겠습니다. 모델 파라미터의 개수와 batch size 등이 GPU메모리에 큰 영향을 미치니 여러가지로 활용해보시기 바랍니다.

Colab의 GPU를 이용하기 위해서, 로컬로 진행하시던 분들도 이번에는 colab을 사용해봅시다. 

- 구글에서 colab의 라이브러리를 찾아서 업로드하세요. 
이후에 Pandas를 이용하여 데이터프레임으로 저장합니다.

colab을 사용하기위해서는 colab 라이브러리들을 잘 활용할 수 있으면 좋습니다. colab 기본형의 GPU의 사용제한 때문에 하지 못하는 일이 있다면, colab pro를 사용할 수 있습니다. 코드스테이츠에서 제공하는 colab pro 설치 가이드가 있지만, 스스로 문제를 해결해보시면 좋습니다. 그러나 도움이 필요하시면 HelpDesk에 문의해주시기 바랍니다.

### 로컬 파일을 업로드하는 코드를 입력하세요. 

In [8]:
from __future__ import print_function

In [1]:
##### Your Code Here #####
from google.colab import files
myfiles = files.upload()
# 데이터 불러오기

Saving TelcomCustomer.csv to TelcomCustomer.csv


In [2]:
import pandas as pd
df = pd.read_csv('TelcomCustomer.csv')

In [3]:
df.head()
df.shape

(7043, 21)

### 문항 2) 결측치가 있는지 isnull()함수를 이용하여 확인하고 결과값을 입력하시오. 

In [4]:
import numpy as np
##### Your Code Here #####
# 결측치 확인
for c in df.columns:
  # print(c, df[c].isnull().sum())
  print(c, df[c].replace(' ',np.nan).isnull().sum())

customerID 0
gender 0
SeniorCitizen 0
Partner 0
Dependents 0
tenure 0
PhoneService 0
MultipleLines 0
InternetService 0
OnlineSecurity 0
OnlineBackup 0
DeviceProtection 0
TechSupport 0
StreamingTV 0
StreamingMovies 0
Contract 0
PaperlessBilling 0
PaymentMethod 0
MonthlyCharges 0
TotalCharges 11
Churn 0


In [5]:
np.nan

nan

### 문항 3) dtypes를 이용해서 데이터 타입을 확인하고 아래 문제에 답하시오.

TotalCharges와 같이 중요한 타켓값이 숫자로 되어있어야 하는데, object로 되어있는 것을 확인하고 숫자형으로 바꿔주세요. <br>
숫자형이 아닌 결측치의 개수는 몇개인가요? 

In [6]:
##### Your Code Here #####
# 데이터 타입 확인
df.info()
# TotalCharges 숫자형으로 변환

# 결측치 확인
df['TotalCharges'].replace(' ', np.nan, inplace=True)
print('빈칸 결측치 수 :',df['TotalCharges'].isnull().sum())
# 결측치 드랍
df.dropna(axis=0, inplace=True)
df.shape

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7043 entries, 0 to 7042
Data columns (total 21 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   customerID        7043 non-null   object 
 1   gender            7043 non-null   object 
 2   SeniorCitizen     7043 non-null   int64  
 3   Partner           7043 non-null   object 
 4   Dependents        7043 non-null   object 
 5   tenure            7043 non-null   int64  
 6   PhoneService      7043 non-null   object 
 7   MultipleLines     7043 non-null   object 
 8   InternetService   7043 non-null   object 
 9   OnlineSecurity    7043 non-null   object 
 10  OnlineBackup      7043 non-null   object 
 11  DeviceProtection  7043 non-null   object 
 12  TechSupport       7043 non-null   object 
 13  StreamingTV       7043 non-null   object 
 14  StreamingMovies   7043 non-null   object 
 15  Contract          7043 non-null   object 
 16  PaperlessBilling  7043 non-null   object 


(7032, 21)

In [7]:
# customerID 드랍
df = df.drop(columns='customerID')

In [8]:
df.shape

(7032, 20)

In [9]:
# Attributing No internet service to No
no_internet_feats = [ 'TechSupport','OnlineBackup', 'DeviceProtection','StreamingTV',
                 'OnlineSecurity','StreamingMovies']

for i in no_internet_feats:
    df[i] = df[i].replace({'No internet service':'No'})

# Attributing No phone service to No
df['MultipleLines']=df['MultipleLines'].replace({'No phone service':'No'})

# Attributing No phone service to No
df['SeniorCitizen']=df['SeniorCitizen'].replace({0:'No',
                                                 1:'Yes'})

In [10]:
!pip install category_encoders

     |████████████████████████████████| 81kB 5.8MB/s 


In [11]:
df.head(3)

,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,OnlineBackup,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn
0,Female,No,Yes,No,1,No,No,DSL,No,Yes,No,No,No,No,Month-to-month,Yes,Electronic check,29.85,29.85,No
1,Male,No,No,No,34,Yes,No,DSL,Yes,No,Yes,No,No,No,One year,No,Mailed check,56.95,1889.5,No
2,Male,No,No,No,2,Yes,No,DSL,Yes,Yes,No,No,No,No,Month-to-month,Yes,Mailed check,53.85,108.15,Yes


In [12]:
# 원핫 인코딩
from category_encoders import OrdinalEncoder

encoder = OrdinalEncoder()
df_encoded = encoder.fit_transform(df)

/usr/local/lib/python3.7/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [13]:
df_encoded.head()

,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,OnlineBackup,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn
0,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,29.85,1,1
1,2,1,2,1,34,2,1,1,2,2,2,1,1,1,2,2,2,56.95,2,1
2,2,1,2,1,2,2,1,1,2,1,1,1,1,1,1,1,2,53.85,3,2
3,2,1,2,1,45,1,1,1,2,2,2,2,1,1,2,2,3,42.30,4,1
4,1,1,2,1,2,2,1,2,1,2,1,1,1,1,1,1,1,70.70,5,2


In [14]:
# 타겟을 0과 1로 바꿔주기
df_encoded['Churn'] = df_encoded['Churn'].replace({1: 0, 2: 1})
df_encoded['Churn'].value_counts()

0    5163
1    1869
Name: Churn, dtype: int64

In [15]:
df_encoded.shape

(7032, 20)

### 문항 4) 훈련집합과 테스트 집합을 나누는 코드를 만들고, 해당 코드를 입력하시오.

- random_state=1
- test_size=0.25

In [16]:
##### Your Code Here #####
# 훈련, 테스트 셋을 나누기
from sklearn.model_selection import train_test_split

train, test = train_test_split(df_encoded, stratify=df_encoded['Churn'], random_state=1, test_size=0.25)
train.shape, test.shape

((5274, 20), (1758, 20))

In [18]:
# features 와 target 을 분리

target = 'Churn'
features = df_encoded.drop(columns=[target]).columns

X_train = train[features]
X_test = test[features]

y_train = train[target]
y_test = test[target]

### 문항 5) sklearn.preprocessing.StandardScaler를 이용하여 정규화를 진행하시고, 문제를 보고 빈칸에 알맞은 단어를 넣으시오.

`X_train_scaled = scaler.##### Your Code Here #####`

In [19]:
##### Your Code Here #####
# 정규화
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test) ##### Your Code Here #####

In [20]:
# 19x1 형태 => 입력층의 노드수가 19개! 
X_train_scaled[0]

array([ 0.99508225, -0.43925252,  0.96350214, -0.65400354, -0.09779796,
        0.32541085,  1.16743446, -1.19019364,  1.59286639,  0.72650531,
       -0.72042294, -0.64249881, -0.79264136,  1.25707392, -0.83276144,
       -0.83448511, -0.27868378, -0.00881575, -1.02585251])

# 모델링

## 기본 모델

In [21]:
# import numpy as np
# from tensorflow.keras.models import Sequential
# from tensorflow.keras.layers import Dense, Dropout, Flatten
# import keras
# import tensorflow as tf
# import IPython
# from tensorflow.keras.wrappers.scikit_learn import KerasClassifier
# from sklearn.model_selection import GridSearchCV

In [22]:
!pip install -U keras-tuner
import kerastuner as kt

     |████████████████████████████████| 71kB 5.4MB/s 
  Created wheel for keras-tuner: filename=keras_tuner-1.0.2-cp37-none-any.whl size=78938 sha256=f466ab00149a8f58bd7aac6b86150e65b99faddc9e432ce950e044049de4a33c
  Stored in directory: /root/.cache/pip/wheels/bb/a1/8a/7c3de0efb3707a1701b36ebbfdbc4e67aedf6d4943a1f463d6
  Created wheel for terminaltables: filename=terminaltables-3.1.0-cp37-none-any.whl size=15356 sha256=c720a613befc282bc51ec3a6d77a5b3447c54b66dc2adcb354e13590b4cf7e49
  Stored in directory: /root/.cache/pip/wheels/30/6b/50/6c75775b681fb36cdfac7f19799888ef9d8813aff9e379663e
Successfully built keras-tuner terminaltables


### 문제 6. np.unique를 이용해서 y_train의 Class 의 개수를 확인하고 입력하시오.

In [23]:
print(np.unique(y_train))
# 이진분류의 경우 class는 2개이지만, 출력층의 노드수 = 1 

[0 1]


In [24]:
# 간단한 모델 만들어서 성능을 보기 !
tf.random.set_seed(7)

model2 = Sequential()
model2.add(Dense(64, activation='relu'))
model2.add(Dense(64, activation='relu'))
model2.add(Dense(1, activation='sigmoid')) # 00분류니까 노드수 1, 활성함수로는 시그모이드

model2.compile(optimizer='adam', 
              loss='binary_crossentropy', 
              metrics=['accuracy'])

results = model2.fit(X_train_scaled, y_train, epochs=10, validation_data=(X_test_scaled,y_test))

Epoch 1/10
165/165 [==============================] - 4s 5ms/step - loss: 0.5132 - accuracy: 0.7363 - val_loss: 0.4490 - val_accuracy: 0.7901
Epoch 2/10
165/165 [==============================] - 0s 3ms/step - loss: 0.4168 - accuracy: 0.8004 - val_loss: 0.4463 - val_accuracy: 0.7929
Epoch 3/10
165/165 [==============================] - 0s 3ms/step - loss: 0.4054 - accuracy: 0.8054 - val_loss: 0.4462 - val_accuracy: 0.7901
Epoch 4/10
165/165 [==============================] - 0s 3ms/step - loss: 0.3978 - accuracy: 0.8055 - val_loss: 0.4446 - val_accuracy: 0.8043
Epoch 5/10
165/165 [==============================] - 0s 3ms/step - loss: 0.3954 - accuracy: 0.8068 - val_loss: 0.4512 - val_accuracy: 0.7947
Epoch 6/10
165/165 [==============================] - 0s 3ms/step - loss: 0.3906 - accuracy: 0.8190 - val_loss: 0.4517 - val_accuracy: 0.7833
Epoch 7/10
165/165 [==============================] - 0s 3ms/step - loss: 0.3889 - accuracy: 0.8217 - val_loss: 0.4517 - val_accuracy: 0.7952
Epoch 

In [25]:
model2.evaluate(X_test_scaled,  y_test, verbose=2) 

55/55 - 0s - loss: 0.4679 - accuracy: 0.7810


[0.4679003953933716, 0.7810011506080627]

In [26]:
# 테스트셋 사용해서 결과 보기
model2.evaluate(X_test_scaled,  y_test, verbose=2) 

55/55 - 0s - loss: 0.4679 - accuracy: 0.7810


[0.4679003953933716, 0.7810011506080627]

파라미터 튜닝을 하기전에 간단히 임의로 넣어본 결과도 꽤 좋다. 이젠 GridSearchCV 를 사용해서 튜닝을 해보겠음!



## GridSearchCV 사용

In [ ]:
# 모델 만들기
tf.random.set_seed(7)

def model_builder(nodes=16, activation='relu'):

  model = Sequential()
  model.add(Dense(nodes, activation=activation))
  model.add(Dense(nodes, activation=activation))
  model.add(Dense(1, activation='sigmoid')) # 이진분류니까 노드수 1, 활성함수로는 시그모이드

  model.compile(optimizer='adam', 
                loss='binary_crossentropy', 
                metrics=['accuracy'])

  return model

# keras.wrapper를 활용하여 분류기를 만듭니다
# keras wrapper가 필수이기 때문에,,
# from tensorflow.keras.wrappers.scikit_learn import KerasClassifier
model = KerasClassifier(build_fn=model_builder, verbose=0)

# GridSearch
batch_size = [50, 100, 300]
epochs = [10, 20, 30]
nodes = [64, 128, 256]
activation = ['relu', 'sigmoid']
param_grid = dict(batch_size=batch_size, epochs=epochs, nodes=nodes, activation=activation)


# GridSearch CV를 만들기
# 3*3*3*2 = 53개 조합, 개별 조합내에서 3 fold CV 53*3 = 162회, 3 fold의 결과 평균을 해당 조합의 퍼포먼스로 사용?(이부분 확인)
grid = GridSearchCV(estimator=model, param_grid=param_grid, cv=3, verbose=1, n_jobs=-1)
grid_result = grid.fit(X_train_scaled, y_train)

Fitting 3 folds for each of 54 candidates, totalling 162 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done  46 tasks      | elapsed:  1.5min
[Parallel(n_jobs=-1)]: Done 162 out of 162 | elapsed:  4.2min finished


In [ ]:
# 최적의 결과값을 낸 파라미터를 출력합니다
print(f"Best: {grid_result.best_score_} using {grid_result.best_params_}")
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print(f"Means: {mean}, Stdev: {stdev} with: {param}")
# Best: 0.8058399756749471 using {'activation': 'sigmoid', 'batch_size': 50, 'epochs': 20, 'nodes': 64}

Best: 0.8060295780499777 using {'activation': 'sigmoid', 'batch_size': 50, 'epochs': 30, 'nodes': 64}
Means: 0.7948426405588785, Stdev: 0.00644115153404292 with: {'activation': 'relu', 'batch_size': 50, 'epochs': 10, 'nodes': 64}
Means: 0.7899127801259359, Stdev: 0.0020942986333580575 with: {'activation': 'relu', 'batch_size': 50, 'epochs': 10, 'nodes': 128}
Means: 0.7902920047442118, Stdev: 0.007508168041281857 with: {'activation': 'relu', 'batch_size': 50, 'epochs': 10, 'nodes': 256}
Means: 0.788395901521047, Stdev: 0.0076738956855800415 with: {'activation': 'relu', 'batch_size': 50, 'epochs': 20, 'nodes': 64}
Means: 0.7846037149429321, Stdev: 0.008312588942436687 with: {'activation': 'relu', 'batch_size': 50, 'epochs': 20, 'nodes': 128}
Means: 0.7692453662554423, Stdev: 0.014922622845349521 with: {'activation': 'relu', 'batch_size': 50, 'epochs': 20, 'nodes': 256}
Means: 0.785362164179484, Stdev: 0.007450459230759025 with: {'activation': 'relu', 'batch_size': 50, 'epochs': 30, 'node

### 문항 7) 최적의 결과를 낸 파라미터와 결과값을 입력하시오.

- batch_size
- epochs
- nodes
- activation

- 정답은 `[100, 100, 100], activation_name` 형태로 입력하시오.

'activation': 'sigmoid', 'batch_size': 100, 'epochs': 30, 'nodes': 128 가 최적으로 나왔다. 정확도는 약 0.80 

이젠 Keras Tuner 를 사용한 파라미터 튜닝도 해보겠음!

## Keras Tuner 사용

In [ ]:
# 모델 만들기

def model_builder(hp):

  model = Sequential()

  # Dense layer에서 노드 수를 조정(32-512)
  hp_units = hp.Int('units', min_value = 32, max_value = 512, step = 32)

  model.add(Dense(units = hp_units, activation='relu'))
  model.add(Dense(units = hp_units, activation='relu'))

  model.add(Dense(1, activation='sigmoid')) # 이진분류니까 노드수 1, 활성함수로는 시그모이드

  # Optimizer의 학습률(learning rate)을 조정[0.01, 0.001, 0.0001]합니다. 
  hp_learning_rate = hp.Choice('learning_rate', values = [1e-2, 1e-3, 1e-4])

  # 컴파일 단계, 옵티마이저와 손실함수, 측정지표를 연결해서 계산 그래프를 구성함
  model.compile(optimizer=keras.optimizers.Adam(learning_rate = hp_learning_rate), 
                loss=keras.losses.BinaryCrossentropy(from_logits = True), 
                metrics=['accuracy'])

  return model

In [ ]:
# 튜너를 인스턴스화하고 하이퍼 튜닝을 수행

tuner = kt.Hyperband(model_builder,
                     objective = 'val_accuracy', 
                     max_epochs = 30, # The maximum number of epochs to train one model. model에 early stopping 써주는 것도 좋음
                     factor = 3,
                     directory = 'my_dir',
                     project_name = 'intro_to_kt')

In [ ]:
# callback 정의 : 하이퍼 파라미터 검색을 실행하기 전에 모든 교육 단계가 끝날 때마다 교육 출력을 지우도록 콜백을 정의합니다.

class ClearTrainingOutput(tf.keras.callbacks.Callback):
  def on_train_end(*args, **kwargs):
    IPython.display.clear_output(wait = True)

In [ ]:
tuner.search(X_train_scaled, y_train, epochs = 30, batch_size=50, validation_data = (X_test_scaled,y_test), callbacks = [ClearTrainingOutput()])

# Get the optimal hyperparameters
best_hps = tuner.get_best_hyperparameters(num_trials = 1)[0]

print(f"""
최적화된 Dense 노드 수 : {best_hps.get('units')} 
최적화된 Learning Rate : {best_hps.get('learning_rate')} 
""")

Trial 62 Complete [00h 00m 01s]
val_accuracy: 0.7986348271369934

Best val_accuracy So Far: 0.8060295581817627
Total elapsed time: 00h 01m 44s
INFO:tensorflow:Oracle triggered exit

최적화된 Dense 노드 수 : 224 
최적화된 Learning Rate : 0.001 



### 문항 8) Keras 튜너를 활용하여 얻어낸 파라미터를 입력하시오. 

- 정답은 `[100, 100]` 형태로 입력하시오.

In [ ]:
X_train.shape[0]/50

105.48

In [ ]:
# 찾은 파라미터들로 모델 만들어보기 (Dense 노드 수 : 416, Learning Rate : 0.01, batch_size : 100, epochs : 30)
from tensorflow.keras import regularizers

tf.random.set_seed(1442)
initializer = tf.keras.initializers.HeNormal()

model = Sequential()

model.add(Dense(best_hps.get('units'), 
                activation='relu', kernel_initializer=initializer,          
                kernel_regularizer=regularizers.l2(0.01),    # L2 norm regularization
                activity_regularizer=regularizers.l1(0.01))) # L1 norm regularization))
model.add(Dense(best_hps.get('units'),
                activation='relu', kernel_initializer=initializer,            
                kernel_regularizer=regularizers.l2(0.01),    # L2 norm regularization
                activity_regularizer=regularizers.l1(0.01)))
model.add(Dense(1, activation='sigmoid')) # 이진분류니까 노드수 1, 활성함수로는 시그모이드

model.compile(optimizer=keras.optimizers.Adam(learning_rate = best_hps.get('learning_rate')), 
              loss='binary_crossentropy', 
              metrics=['accuracy'])

results = model.fit(X_train_scaled, y_train, epochs=2, batch_size=50, validation_data=(X_test_scaled,y_test))

Epoch 1/2
106/106 [==============================] - 1s 5ms/step - loss: 10.0125 - accuracy: 0.7355 - val_loss: 5.8107 - val_accuracy: 0.7947
Epoch 2/2
106/106 [==============================] - 0s 3ms/step - loss: 4.9895 - accuracy: 0.8000 - val_loss: 3.1786 - val_accuracy: 0.7929


In [ ]:
# 테스트셋 사용해서 결과 보기
model.evaluate(X_test_scaled,  y_test, verbose=2) 

55/55 - 0s - loss: 0.4942 - accuracy: 0.8020


[0.49415111541748047, 0.8020477890968323]

# 도전과제

- Random Search 하이퍼 매개 변수 튜닝 구현해보세요.
- hyperos 또는 hyperopts를 이용해서 Bayesian Optimiation tuning 수행 [(링크)](https://https://github.com/maxpumperla/hyperas)
- 기존에 진행했던 강의&프로젝트 데이터셋을 하이퍼 파라미터로 조정해보세요. 
- Cifar100을 이용한다면 90% 이상 달성목표! [참고자료](https://paperswithcode.com/sota/image-classification-on-cifar-100)
- MLP 모델을 forward 와 backward(학습까지)를 처음부터 구현할 수 있는가?
- 케라스에서 MLP 모델을 구현하고 교차 검증(CV)을 통해 하이퍼 파라미터를 조정할 수 있는가?



#### 1) Random Search 하이퍼 매개 변수 튜닝 구현해보세요.

In [28]:
X_train_scaled.shape

(5274, 19)

In [ ]:
def make_model_rscv(nodes=64, func='relu', rate=0.1, l2=0.01):
  model = Sequential()
  model.add(Dense(nodes,
                  activation='relu',
                  input_dim=X_train_scaled.shape[1]))
  model.add(Dense(256, activation=func))
  model.add(Dropout(rate))
  model.add(Dense(512, activation='relu',
            kernel_regularizer=regularizers.l2(l2)))
  model.add(Dense(128, activation='relu'))
  model.add(Dense(32, activation='relu'))
  model.add(Dense(1, activation='sigmoid'))

  model.compile(optimizer = 'adam',
                loss='binary_crossentropy',
                metrics=['accuracy'])

  return model

In [27]:
from sklearn.model_selection import RandomizedSearchCV

In [ ]:
tf.random.set_seed(1442)
param_grid = {'nodes': [32, 128, 256, 512],
              'func':['tanh', 'relu', 'sigmoid', 'softmax'],
              'rate':[0, 0.2, 0.5, 0.7],
              'l2':[0.001, 0.005, 0.01, 0.05, 0.1]}

model_rscv = KerasClassifier(build_fn=make_model_rscv, verbose=0)

grid_rscv = RandomizedSearchCV(
    estimator=model_rscv,
    param_distributions=param_grid,
    n_iter=20,
    cv=3,
    scoring='accuracy',
    verbose=3,
    n_jobs=1,
    random_state=12)

grid_result_rscv = grid_rscv.fit(X_train_scaled, y_train)

Fitting 3 folds for each of 20 candidates, totalling 60 fits
[CV] rate=0.7, nodes=512, l2=0.005, func=sigmoid .....................


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/engine/sequential.py:450: UserWarning: `model.predict_classes()` is deprecated and will be removed after 2021-01-01. Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).
  warnings.warn('`model.predict_classes()` is deprecated and '
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    1.0s remaining:    0.0s


[CV]  rate=0.7, nodes=512, l2=0.005, func=sigmoid, score=0.794, total=   1.0s
[CV] rate=0.7, nodes=512, l2=0.005, func=sigmoid .....................
[CV]  rate=0.7, nodes=512, l2=0.005, func=sigmoid, score=0.792, total=   0.9s
[CV] rate=0.7, nodes=512, l2=0.005, func=sigmoid .....................


[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    1.9s remaining:    0.0s


[CV]  rate=0.7, nodes=512, l2=0.005, func=sigmoid, score=0.792, total=   0.9s
[CV] rate=0.2, nodes=32, l2=0.001, func=relu .........................
[CV]  rate=0.2, nodes=32, l2=0.001, func=relu, score=0.783, total=   1.1s
[CV] rate=0.2, nodes=32, l2=0.001, func=relu .........................
[CV]  rate=0.2, nodes=32, l2=0.001, func=relu, score=0.783, total=   0.9s
[CV] rate=0.2, nodes=32, l2=0.001, func=relu .........................
[CV]  rate=0.2, nodes=32, l2=0.001, func=relu, score=0.795, total=   0.9s
[CV] rate=0.2, nodes=256, l2=0.05, func=sigmoid ......................
[CV]  rate=0.2, nodes=256, l2=0.05, func=sigmoid, score=0.760, total=   0.9s
[CV] rate=0.2, nodes=256, l2=0.05, func=sigmoid ......................
[CV]  rate=0.2, nodes=256, l2=0.05, func=sigmoid, score=0.796, total=   0.9s
[CV] rate=0.2, nodes=256, l2=0.05, func=sigmoid ......................
[CV]  rate=0.2, nodes=256, l2=0.05, func=sigmoid, score=0.781, total=   0.9s
[CV] rate=0.5, nodes=512, l2=0.001, func=si

[Parallel(n_jobs=1)]: Done  60 out of  60 | elapsed:   57.0s finished


In [ ]:
rs = pd.DataFrame(grid_result_rscv.cv_results_).sort_values(by='rank_test_score').head()
rs.T

,3,5,16,12,7
mean_fit_time,0.792805,0.794649,0.796603,0.880406,0.821169
std_fit_time,0.0110927,0.0124274,0.00341146,0.124761,0.0113597
mean_score_time,0.11431,0.112394,0.109338,0.110921,0.116589
std_score_time,0.00230824,0.000843507,0.000459571,0.00400837,0.00647866
param_rate,0.5,0.2,0.2,0.5,0
param_nodes,512,256,256,128,512
param_l2,0.001,0.001,0.01,0.001,0.001
param_func,sigmoid,sigmoid,tanh,relu,tanh
params,"{'rate': 0.5, 'nodes': 512, 'l2': 0.001, 'func...","{'rate': 0.2, 'nodes': 256, 'l2': 0.001, 'func...","{'rate': 0.2, 'nodes': 256, 'l2': 0.01, 'func'...","{'rate': 0.5, 'nodes': 128, 'l2': 0.001, 'func...","{'rate': 0, 'nodes': 512, 'l2': 0.001, 'func':..."
split0_test_score,0.801479,0.807736,0.799772,0.800341,0.798066


RandomizedSearchCV를 썼을 때의 결과도 GridSearchCV를 사용했을 때와 다르게 나왔다. 하지만 시간 때문에 RandomizedSearchCV의 iter수를 탐색 영역에 비해서 작게 설정했기 때문에 위 결과가 최선의 결과라고 할 수는 없다.

### 2) hyperos 또는 hyperopts를 이용해서 Bayesian Optimiation tuning 수행

In [ ]:
!pip install hyperas
!pip install category_encoders

In [11]:

import numpy as np

from hyperopt import Trials, STATUS_OK, tpe
from keras.datasets import mnist
from keras.layers.core import Dense, Dropout, Activation
from keras.models import Sequential
from keras.utils import np_utils

from hyperas import optim
from hyperas.distributions import choice, uniform

import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from category_encoders import OrdinalEncoder

In [2]:
# hyperas 를 colab에서 사용하려고 하면, 노트북을 찾지 못해서 코드가 돌아가지 않는다.
# from hyperopt import Trials, Trials class에서 경로를 받아서 결과를 기록하는 것 같은데
# 해당 부분이 돌아가지 않음, colab을 지금 돌아가고 있는 환경에 monunt해야 하는 것 같음
# 노트북이 내 google driver에 저장되어 있어야함
# see: https://towardsdatascience.com/keras-hyperparameter-tuning-in-google-colab-using-hyperas-624fa4bbf673
# See: https://stackoverflow.com/questions/49920031/get-the-path-of-the-notebook-on-google-colab
# Install the PyDrive wrapper & import libraries.
!pip install -U -q PyDrive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

# Authenticate and create the PyDrive client.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

# Copy/download the file
fid = drive.ListFile({'q':"title='ds-cs-N424a_.ipynb'"}).GetList()[0]['id']
f = drive.CreateFile({'id': fid})
f.GetContentFile('ds-cs-N424a_.ipynb')

In [5]:


# data load 부분을 함수로 다 구현해야함..?
def data():

  df = pd.read_csv('TelcomCustomer.csv')
  df['TotalCharges'].replace(' ', np.nan, inplace=True)
  df.dropna(axis=0, inplace=True)
  df = df.drop(columns='customerID')
  # Attributing No internet service to No
  no_internet_feats = [ 'TechSupport','OnlineBackup', 'DeviceProtection','StreamingTV',
                  'OnlineSecurity','StreamingMovies']

  for i in no_internet_feats:
    df[i] = df[i].replace({'No internet service':'No'})

  # Attributing No phone service to No
  df['MultipleLines']=df['MultipleLines'].replace({'No phone service':'No'})

  # Attributing No phone service to No
  df['SeniorCitizen']=df['SeniorCitizen'].replace({0:'No',
                                                  1:'Yes'})
  encoder = OrdinalEncoder()
  df_encoded = encoder.fit_transform(df)

  # 타겟을 0과 1로 바꿔주기
  df_encoded['Churn'] = df_encoded['Churn'].replace({1: 0, 2: 1})
  df_encoded['Churn'].value_counts()

  train, test = train_test_split(df_encoded, stratify=df_encoded['Churn'], random_state=1, test_size=0.25)

  target = 'Churn'
  features = df_encoded.drop(columns=[target]).columns

  X_train = train[features]
  X_test = test[features]

  y_train = train[target]
  y_test = test[target]

  scaler = StandardScaler()
  X_train_scaled = scaler.fit_transform(X_train)
  X_test_scaled = scaler.transform(X_test)

  X_train_ = X_train_scaled
  X_test_ = X_test_scaled
  y_train_ = y_train
  y_test_ = y_test

  return X_train_, y_train_, X_test_, y_test_

def create_model(X_train_, y_train_, X_test_, y_test_):

  model = Sequential()
  model.add(Dense({{choice([256, 512, 1024])}}, input_shape=(19,)) ) # {{}}사이에 넣고 choice
  model.add(Activation({{choice(['relu', 'sigmoid'])}}))
  # model.add(Dropout({{uniform(0, 1)}})) # Dropout 조절가능
  model.add(Dense({{choice([256, 512, 1024])}}))
  model.add(Activation({{choice(['relu', 'sigmoid'])}}))
  model.add(Dense(1, activation='sigmoid')) # 2진분류

  model.compile(loss='binary_crossentropy',
                metrics=['accuracy'],
                optimizer={{choice(['adam', 'sgd'])}})

  result = model.fit(X_train_, y_train_,
                     batch_size= {{choice([32,64])}},
                     epochs=3,
                     verbose=2,
                     validation_split=0.2)
  
  #get the highest validation accuracy of the training epochs
  #아마도 traing set 속에서 validation_split 0.2퍼 하고 acc amax를 저장하는 것 같음
  # print(result.history)
  validation_acc = np.amax(result.history['val_accuracy']) # 공식 깃 예제에 잘못되있음,, val_acc 아님,,
  print('Best validation acc of epoch:', validation_acc)
  return {'loss': -validation_acc, 'status': STATUS_OK, 'model': model}
  
best_run, best_model = optim.minimize(model=create_model,
                                          data=data,
                                          algo=tpe.suggest,
                                          max_evals=5,
                                          trials=Trials(),
                                          notebook_name='ds-cs-N424a_'
                                          )



>>> Imports:
#coding=utf-8

from __future__ import print_function

try:
    from google.colab import files
except:
    pass

try:
    import pandas as pd
except:
    pass

try:
    import numpy as np
except:
    pass

try:
    from category_encoders import OrdinalEncoder
except:
    pass

try:
    from sklearn.model_selection import train_test_split
except:
    pass

try:
    from sklearn.preprocessing import StandardScaler
except:
    pass

try:
    import kerastuner as kt
except:
    pass

try:
    from tensorflow.keras import regularizers
except:
    pass

try:
    from sklearn.model_selection import RandomizedSearchCV
except:
    pass

try:
    import numpy as np
except:
    pass

try:
    from hyperopt import Trials, STATUS_OK, tpe
except:
    pass

try:
    from keras.datasets import mnist
except:
    pass

try:
    from keras.layers.core import Dense, Dropout, Activation
except:
    pass

try:
    from keras.models import Sequential
except:
    pass

try:
    from keras.utils im

In [6]:
best_run, best_model # chioce index 번호로 저장되어 있음

({'Activation': 0,
  'Activation_1': 0,
  'Dense': 1,
  'Dense_1': 2,
  'Dropout': 0.9128294469805703,
  'batch_size': 0,
  'optimizer': 0},
 <tensorflow.python.keras.engine.sequential.Sequential at 0x7f03d06c8c90>)

In [12]:
_, _, X_test, Y_test = data() # 근데 이렇게 하면 다시 섞일거 같은데 시드가 1이면 항상 같은가?
best_model.evaluate(X_test, Y_test)


55/55 [==============================] - 0s 2ms/step - loss: 0.4710 - accuracy: 0.8003


[0.4710337221622467, 0.8003413081169128]

***Keras + Hyperopt: A very simple wrapper for convenient hyperparameter optimization!!!!***

매우 간단하다는 말에 속아 일단 시작했다.(우여곡절이 있었지만 아무튼 한두시간 정도에 간단하게 적용해볼 수 있었으니까 Simple인가,,,?). 구체적으로 어떻게 동작하는지는 좀더 문서들을 읽어봐야겠지만, 매우 빠르게 학습이 진행되는 느낌은 있다(그래서 베이지안인가?). 하지만 keras tuner가 더 쉬운거 같은데,,  
원하는 코드들을 함수형태로 다 만든다음에 블럭처럼,? optim.minimize 속에 넣어주고, trials=Trials() 이부분을 통해서 새로운 .py를 알아서 만들어서, 그 속에서 돌아가는 것처럼 생각된다. print를 보면 노트북 속의 코드들이 전부 찍혀 나오는 것을 볼 수 있다.  
아무튼 적당히 코드를 적용하게 만들어 두었으니 나중에 써먹을 수 있을 것 같다. 일단 장점과 좀더 구체적인 동작방식을 알아야 할 것 같다.

### 3) Clfar 100을 이용한 이전 코드를 튜닝
---
이부분은 어제 도전과제에서 조금 해본 부분이 있다.

In [7]:
# 그냥 런타임 초기화 할것.

import keras
import tensorflow as tf
import numpy as np

from tensorflow.keras.datasets import cifar100

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, BatchNormalization
from tensorflow.keras.layers import Dropout

from tensorflow.keras.optimizers import Adam, SGD
from tensorflow.keras import regularizers
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.utils import to_categorical

from keras.applications import VGG19, ResNet50

from sklearn.model_selection import train_test_split

from kerastuner.tuners import RandomSearch, Hyperband


In [6]:
(X_train, y_train), (X_test, y_test) = cifar100.load_data()

169009152/169001437 [==============================] - 4s 0us/step


In [ ]:
# nomalize
X_train = X_train.astype('float32') / 255.0 - 0.5
X_test = X_test.astype('float32') / 255.0 - 0.5


# validation set 분리
X_train, X_val, y_train, y_val= train_test_split(X_train, y_train, test_size=.2)

# one hot encording target value
y_train=to_categorical(y_train)
y_val=to_categorical(y_val)
y_test=to_categorical(y_test)

In [ ]:
def 
# VGG 19는 pre-define된 convolution layer
base_model_1 = VGG19(include_top=False, weights='imagenet', input_shape=(32,32,3), classes=y_train.shape[1])

model_1= Sequential()
model_1.add(base_model_1) 
model_1.add(Flatten()) 

#Add the Dense layers along with activation and batch normalization
model_1.add(Dense(1024,activation=('relu'),input_dim=512))
model_1.add(Dense(512,activation=('relu'))) 
model_1.add(Dense(256,activation=('relu'))) 
# model_1.add(Dropout(.3)) #Adding a dropout layer that will randomly drop 30% of the weights
model_1.add(Dense(128,activation=('relu')))
# model_1.add(Dropout(.2))
model_1.add(Dense(num_classes,activation=('softmax'))) #This is the classification layer

model_1.summary()

In [ ]:
learn_rate=.001

sgd=SGD(lr=learn_rate,momentum=.9,nesterov=False)
adam=Adam(lr=learn_rate, beta_1=0.9, beta_2=0.999, epsilon=None, decay=0.0, amsgrad=False)

model_1.compile(optimizer=sgd,loss='categorical_crossentropy',metrics=['accuracy'])

model_1.fit(X_train, y_train, batch_size=batch_size,
                    validation_data=(X_val, y_val),
                    epochs=epochs, verbose=1)

### 4) MLP 모델을 forward 와 backward(학습까지)를 처음부터 구현할 수 있는가?

In [ ]:
from math import sqrt

In [ ]:
class MLP:
  def __init__(self, input_nodes, hidden_nodes, output_nodes, lr):
    """
    MLP 클래스의 생성자 함수입니다.
    
    Args:
        input_nodes: 입력층의 노드 개수
        hidden_nodes: 은닉층의 노드 개수
        output_nodes: 출력층의 노드 개수
        lr: 학습률
    """
    self.input_nodes = input_nodes
    self.hidden_nodes = hidden_nodes
    self.output_nodes = output_nodes
    self.lr = lr

    self.w_ih = np.random.normal(0,
                                 sqrt(2/(self.input_nodes + self.hidden_nodes)),
                                 (self.input_nodes, self.hidden_nodes))
    self.w_ho = np.random.normal(0,
                                 sqrt(2/(self.hidden_nodes + self.output_nodes)),
                                 (self.hidden_nodes, self.output_nodes))
    

  def sigmoid(self, x):
    """
    활성화 함수로 사용할 sigmoid 함수입니다.
    """
    return 1/(1+np.exp(-x))


  def fit(self, X_train, y_train):
    """
    모델을 학습합니다.

    Args:
        X_train(2차원 numpy.array, float): 학습할 데이터
        y_train(2차원 numpy.array, float): 학습할 데이터의 타겟
    # """
    self.inputs = X_train
    self.targets = y_train
    return self.forward('train')


  def forward(self, filter):
    if filter == 'train':
      self.h = np.dot(self.inputs, self.w_ih)
      self.H = self.sigmoid(self.h)

      self.y = np.dot(self.H, self.w_ho)
      self.Y = self.sigmoid(self.y)

      return self.backward()

    elif filter == 'pred':
      self.h = np.dot(self.inputs, self.w_ih)
      self.H = self.sigmoid(self.h)

      self.y = np.dot(self.H, self.w_ho)
      self.Y = self.sigmoid(self.y)
      return self.Y

  # 가중치 업데이트
  def backward(self):
    self.o_error = -(self.targets-self.Y)
    self.o_delta = self.o_error * (self.Y * (1 - self.Y))

    self.h_error = self.o_delta.dot(self.w_ho.T)
    self.h_delta = self.h_error * (self.y * (1 - self.y))

    self.w_ih = self.w_ih - self.lr*self.inputs.T.dot(self.h_delta)
    self.w_ho = self.w_ho - self.lr*self.H.T.dot(self.o_delta)


  def predict(self, X_test):
    """
    타겟을 예측하는 함수입니다.
    """
    self.inputs = X_test

    result = self.forward('pred')
    return result

In [ ]:
a = np.array([[1, 2], [3, 4], [5, 6]])
b = np.array([[10], [20], [30]])

In [ ]:
mlp = MLP(2, 3, 1, 0.01)

In [ ]:
# 초기 가중치
print(mlp.w_ih)
print(mlp.w_ho)

[[-0.6125871   0.77772355  0.12272781]
 [-0.10692782 -0.581819   -0.54900859]]
[[-0.25205016]
 [-0.14311679]
 [ 0.04712651]]


In [ ]:
mlp.fit(a, b)

In [ ]:
# 학습 후 가중치
print(mlp.w_ih)
print(mlp.w_ho)

[[-0.59924549  0.78529906  0.12023329]
 [-0.08978862 -0.57208718 -0.55221316]]
[[-0.23847518]
 [-0.06502462]
 [ 0.06509709]]


In [ ]:
c = np.array([[10, 12]])

In [ ]:
mlp.predict(c)

array([[0.48817742]])